## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")


In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="llama-3.1-8b-instant",groq_api_key=groq_api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000287E26414E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000287E2641BD0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])

AIMessage(content="Nice to meet you, Krish! It's great to hear that you're a Chief AI Engineer. That's a fascinating field that's constantly evolving and innovating. What areas of AI are you most interested in or work on, and what challenges are you currently facing in your role?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 48, 'total_tokens': 106, 'completion_time': 0.275497108, 'completion_tokens_details': None, 'prompt_time': 0.004953893, 'prompt_tokens_details': None, 'queue_time': 0.345235395, 'total_time': 0.280451001}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd6ff-d059-7ee0-b05c-a603dbff599c-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 48, 'output_tokens': 58, 'total_tokens': 106})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="Your name is Krish, and you're a Chief AI Engineer.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 117, 'total_tokens': 131, 'completion_time': 0.018382831, 'completion_tokens_details': None, 'prompt_time': 0.01029775, 'prompt_tokens_details': None, 'queue_time': 0.094311669, 'total_time': 0.028680581}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd700-0bc1-78a2-afa6-de3c32f0ee94-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 117, 'output_tokens': 14, 'total_tokens': 131})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [7]:
config={"configurable":{"session_id":"chat1"}}

In [8]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")],
    config=config
)

In [9]:
response.content

"Nice to meet you, Krish. As a Chief AI Engineer, I'm sure you have a deep understanding of the latest advancements in artificial intelligence and machine learning. What specific areas of AI do you specialize in, and what projects or initiatives are you currently working on? Are you looking to explore any new technologies or concepts, or do you have any questions you'd like to discuss?"

In [10]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content="Your name is Krish. You're a Chief AI Engineer.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 139, 'total_tokens': 152, 'completion_time': 0.028047611, 'completion_tokens_details': None, 'prompt_time': 0.03273131, 'prompt_tokens_details': None, 'queue_time': 0.322287471, 'total_time': 0.060778921}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_020e283281', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd778-639b-73c0-baa3-04dacc9c695a-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 139, 'output_tokens': 13, 'total_tokens': 152})

In [11]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"I don't have any information about your name. I'm a large language model, I don't have the ability to retain information about individual users or their personal details. Each time you interact with me, it's a new conversation and I don't retain any context from previous conversations. If you'd like to share your name with me, I can use it in our conversation."

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Nice to meet you, John. I'll keep that in mind for our conversation. Is there something specific you'd like to talk about or ask, or are you just looking for some general conversation?"

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John.'